## VIEW
### VIEW : 뷰는 DB에 존재하는 가상 테이블
- 실제 테이블처럼 행과 열을 가지고 있지만, 실제로 데이터를 저장하진 않음
- MySQL에서 뷰는 다른 테이블이나 다른 뷰에 저장되어 있는 데잍러르 보여주는 역할만 수행
- 뷰를 사용하면 여러 테이블이나 뷰를 하나의 테이블처럼 볼 수 있음
- 뷰의 장점 : 특정 사용자에게 테이블 전체가 아닌 필요한 컬렄만 보여줌, 복잡한 쿼리를 단순화, 쿼리 재사용 가능
- 뷰의 단점 : 한번 정의된 뷰는 변경 불가, 삽입/삭제/갱신 작업에 많은 제약, 자신만의 인덱스를 가질 수 없음

- **단순 뷰 : 하나의 테이블로 만든 뷰 (수정가능)**
- **복합 뷰 : 두 개 이상의 테이블로 만든 뷰 (익기전용)** 
- **뷰가 참조하는 테이블 삭제시 뷰는 조회 불가 (뷰가 조회되지 않으면 CHECK TABLE 문으로 뷰의 상태 확인 가능)**

### ```CREATE VIEW AS``` : 뷰 생성
- 테이블과 동일하게 select문을 사용하여 접근
- 조건식도 추가 가능

**city, country, countrylanguage 테이블을 JOIN하고, 한국에 대한 정보만 뷰 생성**

In [38]:
dbGetQuery(conn, "CREATE VIEW allView 
                  AS 
                  SELECT city.Name, country.SurfaceArea, city.Population, countrylanguage.language
                  FROM city
                  JOIN country ON city.CountryCode = country.CODE
                  JOIN countrylanguage ON city.CountryCode = countrylanguage.CountryCode
                  WHERE city.CountryCode = 'KOR';")

<0 x 0 matrix>

In [31]:
dbGetQuery(conn, "SELECT * FROM allView;")

,Name,SurfaceArea,Population,language
,<chr>,<dbl>,<int>,<chr>
1,Seoul,99434,9981619,Chinese
2,Pusan,99434,3804522,Chinese
3,Inchon,99434,2559424,Chinese


### ```ALTER VIEW``` : 뷰의 수정

In [34]:
dbGetQuery(conn, "ALTER VIEW allView 
                  AS
                  SELECT mem_id FROM market_db.member;")

<0 x 0 matrix>

In [37]:
dbGetQuery(conn, "SELECT * FROM allView;") 

,mem_id
,<chr>
1,BLK
2,MMU
3,RED
4,ITZ
5,APN


### ```DESCRIBE {뷰_이름}``` : 뷰의 정보 확인

In [32]:
dbGetQuery(conn, "DESC allVIew;")

Field,Type,Null,Key,Default,Extra
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Name,char(35),NO,,,
SurfaceArea,"float(10,2)",NO,,0.00,
Population,int,NO,,0,
language,char(30),NO,,,


### ```SHOW CREATE VIEW {뷰_이름}``` : 뷰의 소스코드 확인

In [33]:
dbGetQuery(conn, "SHOW CREATE VIEW allView;")

View,Create View,character_set_client,collation_connection
<chr>,<chr>,<chr>,<chr>
allview,"CREATE ALGORITHM=UNDEFINED DEFINER=`root`@`localhost` SQL SECURITY DEFINER VIEW `allview` AS select `city`.`Name` AS `Name`,`country`.`SurfaceArea` AS `SurfaceArea`,`city`.`Population` AS `Population`,`countrylanguage`.`Language` AS `language` from ((`city` join `country` on((`city`.`CountryCode` = `country`.`Code`))) join `countrylanguage` on((`city`.`CountryCode` = `countrylanguage`.`CountryCode`))) where (`city`.`CountryCode` = 'KOR')",utf8mb4,utf8mb4_general_ci


### 뷰를 통한 데이터의 수정/삭제
- 뷰를 통하여 데이터를 입력하려면, 뷰에서 보이지 안흔 테이블의 열에 NOT NULL이 없어야 함

### ```UPDATE {뷰_이름}```

**allView 뷰에서 Name이 'Pusan'인 데이터의 Population을 0으로 수정**

In [44]:
dbGetQuery(conn, "UPDATE allView 
                  SET Population = 0
                  WHERE Name = 'Pusan';")

<0 x 0 matrix>

In [46]:
dbGetQuery(conn, "SELECT * FROM allView
                  WHERE Name = 'Pusan';")

,Name,SurfaceArea,Population,language
,<chr>,<dbl>,<int>,<chr>
1,Pusan,99434,0,Chinese


### ```DELETE FROM {뷰_이름}``` 

### ```INSERT INTO {뷰_이름} VALUES ~``` : 뷰를 통한 데이터 입력

**뷰는 price가 50이상인 데이터만 표시**

### ```WITH CHECK OPTION``` : price가 50이상이라는 조건을 만족하지 못하는 값은 입력되지 않도록 설정

In [115]:
dbGetQuery(conn, "CREATE OR REPLACE VIEW price50
                  AS 
                  SELECT mem_id, price, amount
                  FROM market_db.buy
                  WHERE price >= 50
                  WITH CHECK OPTION;")

<0 x 0 matrix>

In [116]:
dbGetQuery(conn, "SELECT * FROM price50;")

mem_id,price,amount
<chr>,<int>,<int>
BLK,1000,1
BLK,50,3
MMU,80,10
APN,50,1


**mem_id가 'APN'인 데이터 삭제**

In [117]:
dbGetQuery(conn, "DELETE FROM price50
                  WHERE mem_id = 'APN';")

<0 x 0 matrix>

In [118]:
dbGetQuery(conn, "SELECT * FROM price50;")

mem_id,price,amount
<chr>,<int>,<int>
BLK,1000,1
BLK,50,3
MMU,80,10


### ```WITH CHECK OPTION```로 인하여 price가 100이하인 값 입력 불가

In [119]:
dbGetQuery(conn, "INSERT INTO price50 
                  VALUES('AAA', 49, 1);")

ERROR: Error in .local(conn, statement, ...): could not run statement: Field of view 'world.price50' underlying table doesn't have a default value


### 뷰가 참조하는 테이블 삭제

In [120]:
dbGetQuery(conn, "DROP TABLE IF EXISTS market_db.buy;")

<0 x 0 matrix>

**참조되는 테이블이 삭제되어 뷰 확인 불가**

In [121]:
dbGetQuery(conn, "SELECT * FROM price50;")

ERROR: Error in .local(conn, statement, ...): could not run statement: View 'world.price50' references invalid table(s) or column(s) or function(s) or definer/invoker of view lack rights to use them


**CHECK TABLE 문으로 뷰의 상태 확인**

In [122]:
dbGetQuery(conn, "CHECK TABLE price50;")

Table,Op,Msg_type,Msg_text
<chr>,<chr>,<chr>,<chr>
world.price50,check,Error,View 'world.price50' references invalid table(s) or column(s) or function(s) or definer/invoker of view lack rights to use them
world.price50,check,error,Corrupt


### ```DROP VIEW``` : 뷰의 삭제

In [123]:
dbGetQuery(conn, "DROP VIEW price100;")

<0 x 0 matrix>

#  

## 스토어드 프로시저 (STORED PROCEDURE)
### 스토어드 프로시저 (STORED PROCEDURE) : 함수/단축키, 여러 개의 SQL문을 하나로 묶음
- 매개변수 사용가능

### ```CREATE PROCEDURE {스토어드_프로시저} ~ ``` : 스토어드 프로시저 생성

```SQL
DELIMITER $$
CREATE PROCEDURE {스토어드_프로시저} (
    IN {입력 매개변수} {데이터_형식},
    OUT {출력 매개변수} {데이터_형식}
BEGIN

~ 

END $$
DELIMITER ;
```

### ```CALL {스토어드_프로시저}(입력값)``` : 스토어드 프로시저 실행
```SQL
CALL {스토어드_프로시저}(입력값);
```

### ```DROP PROCEDURE {스토어드_프로시저}``` : 스토어드 프로시저 삭제
```SQL
DROP PROCEDURE IF EXISTS {스토어드 프로시저};
```

#  
### 스토어드 프로시저 활용

**성별을 입력하면, 데이터 내 성별에 해당하는 데이터의 개수를 반환하는 스토어드 프로시저 생성**

```SQL
DROP PROCEDURE IF EXISTS SSOL.user_proc1;
DELIMITER $$
CREATE PROCEDURE SSOL.user_proc1(IN SEX_input CHAR(10))
BEGIN
	SELECT Sex, count(*) AS n FROM heart 
    GROUP BY Sex 
    HAVING Sex = SEX_input;
END $$
DELIMITER ;
```

In [149]:
dbGetQuery(conn, "CALL SSOL.user_proc1('M');")

Sex,n
<chr>,<int>
M,725


In [153]:
dbGetQuery(conn, "CALL SSOL.user_proc1('F');")

Sex,n
<chr>,<int>
F,193


**매개변수가 2개인 함수** : 성별과 나이를 입력

```SQL
DROP PROCEDURE IF EXISTS SSOL.user_proc3;
DELIMITER $$
CREATE PROCEDURE SSOL.user_proc3(
	IN SEX_input CHAR(10),
    IN Age_input INT(10)
    )
BEGIN
	SELECT Age, Sex, count(*) AS n FROM heart 
    GROUP BY Age, Sex 
    HAVING Sex = SEX_input AND Age > Age_input;
END $$
DELIMITER ;

```

In [158]:
dbGetQuery(conn, "CALL SSOL.user_proc3('F', 70);")

Age,Sex,n
<int>,<chr>,<int>
71,F,3
73,F,1
74,F,1
76,F,1


## 
### 출력 매개변수의 활용

**noTable이라는 이름의 테이블에 넘겨 받은 값을 입력**

**id열의 최대 값을 알아내는 기능**

**id열의 최대값은 결국 방금 입력한 행의 순차 번호**

```SQL    
DELIMITER $$    
CREATE PROCEDURE SSOL.user_proc4(
    IN txtValue CHAR(10),
    OUT outValue INT     )
BEGIN
  INSERT INTO noTable VALUES(NULL, txtValue);
  SELECT MAX(id) INTO outValue FROM noTable; 
END $$
DELIMITER ; 
```

```SQL
CREATE TABLE IF NOT EXISTS noTable(
    id INT AUTO_INCREMENT PRIMARY KEY, 
    txt CHAR(10));
```

In [164]:
dbGetQuery(conn, "desc noTable;")

Field,Type,Null,Key,Default,Extra
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
id,int,NO,PRI,NA,auto_increment
txt,char(10),YES,,NA,


- noTable의 txt열에 값들을 저장하고, 테이블의 id열 최댓값을 @myValue에 저장 

In [169]:
dbGetQuery(conn, "CALL SSOL.user_proc4 ('A', @myValue);")
dbGetQuery(conn, "CALL SSOL.user_proc4 ('B', @myValue);")
dbGetQuery(conn, "CALL SSOL.user_proc4 ('C', @myValue);")
dbGetQuery(conn, "CALL SSOL.user_proc4 ('D', @myValue);")

<0 x 0 matrix>

<0 x 0 matrix>

In [170]:
dbGetQuery(conn, "SELECT CONCAT('INPUT ID Value =>', @myValue);")

"CONCAT('INPUT ID Value =>', @myValue)"
<chr>
INPUT ID Value =>4


## 

### SQL 프로그래밍 활용

### ```DECLARE``` : 지역변수 생성


### ```IF ``` : IF 조건문

```SQL
IF {조건1} THEN {SQL문1}
ELSE {SQL문2}
END IF;
```

- debutDate, curDate, days 날짜 지역 변수 생성
- member테이블에서 mem_id가 ‘APN’인 데이터의 debut_date를 debutDate에 저장
- curDATE를 현재 날짜로 설정, days를 debutDate부터 curDATE까지 지난 일 수로 저장
- days/365 > 5 조건을 만족하는지 여부에 따라 메시지 생성하는 스토어드 프로시져 생성

In [220]:
dbGetQuery(conn, "SELECT * FROM market_db.member")

,mem_id,mem_name,mem_number,addr,phone1,phone2,height,debut_date
,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<chr>
1,APN,에이핑크,6,경기,031,77777777,164,2011-02-10
2,BLK,블랙핑크,4,경남,055,22222222,163,2016-08-08
3,GRL,소녀시대,8,서울,02,44444444,168,2007-08-02
4,ITZ,잇지,5,경남,NA,NA,167,2019-02-12
5,MMU,마마무,4,전남,061,99999999,165,2014-06-19
6,OMY,오마이걸,7,서울,NA,NA,160,2015-04-21


```SQL
DROP PROCEDURE IF EXISTS ifProc3;
DELIMITER $$
CREATE PROCEDURE ifProc3()
BEGIN
    DECLARE debutDate DATE; 
    DECLARE curDate DATE; 
    DECLARE days INT; 
    SELECT debut_date INTO debutDate 
       FROM market_db.member
       WHERE mem_id = 'APN';
    SET curDATE = CURRENT_DATE(); 
    SET days =  DATEDIFF(curDATE, debutDate); 
    IF (days/365) >= 5 THEN -- 5년이 지났다면
          SELECT CONCAT('It has been ', days, 'days.');
    ELSE
          SELECT 'It has been only ' + days + 'days.' ;
    END IF;
END $$
DELIMITER ;
```

In [212]:
dbGetQuery(conn, "CALL market_db.ifProc3(); ")

"CONCAT('It has been ', days, 'days.')"
<chr>
It has been 4155days.


# 

### ```CASE``` : CASE 조건문
```SQL
CASE
WHEN {조건1} THEN {SQL문1}
WHEN {조건2} THEN {SQL문2}
~ 
ELSE {SQL문}
END CASE;
```
- case문의 결과는 select문의 호출 대상이 될 수 있다


**buy B 테이블과 member M 테이블에서, member M을 기준으로 결합하고**

**결합한 데이터를 M.mem_id별 M.mem_id, M.mem_name, SUM(price*amount) (Total), CASE문에 따른 ‘Grade’를, Total의 내림차순으로 호출**

```SQL
CREATE TABLE Grade 
AS
SELECT M.mem_id, M.mem_name, SUM(price*amount) AS Total,
        CASE  
           WHEN (SUM(price*amount)  >= 1500) THEN 'A grade'
           WHEN (SUM(price*amount)  >= 1000) THEN 'B grade'
           WHEN (SUM(price*amount) >= 1 ) THEN 'C grade'
           ELSE 'D grade'
        END "Grade"
   FROM buy B
         RIGHT OUTER JOIN member M
         ON B.mem_id = M.mem_id
   GROUP BY M.mem_id
   ORDER BY Total DESC ; 
```

In [222]:
dbGetQuery(conn, "select mem_id, Total, Grade from market_db.Grade;")

Warning message in .local(conn, statement, ...):
"Decimal MySQL column 1 imported as numeric"


mem_id,Total,Grade
<chr>,<dbl>,<chr>
MMU,1950,A grade
BLK,1210,B grade
APN,295,C grade
GRL,75,C grade
ITZ,NA,D grade
OMY,NA,D grade
RED,NA,D grade
SPC,NA,D grade
TWC,NA,D grade


# 

### ```WHILE``` : 반복문
```SQL
WHILE {조건문} DO {SQL문} END WHILE;
```
- <조건문>이 참인 동안 반복

**1부터 100까지의 합 계산**

```SQL
DROP PROCEDURE IF EXISTS whileProc;

DELIMITER $$
CREATE PROCEDURE whileProc()
BEGIN
    DECLARE i INT; 
    DECLARE hap INT; 
    SET i = 1;
    SET hap = 0;
    WHILE (i <= 100) DO
        SET hap = hap + i;  
        SET i = i + 1;     
    END WHILE;
    SELECT 'Sum of 1 to 100 ==>', hap;   
END $$
DELIMITER ;
```

In [230]:
dbGetQuery(conn, "CALL whileProc();")

Sum of 1 to 100 ==>,hap
<chr>,<int>
Sum of 1 to 100 ==>,5050


# 

### ```ITERATE``` : 지정한 레이블로 가서 계속 진행

### ```LEAVE``` : 지정한 레이블 탈출, WHILE문 종료

**4의 배수를 제외한 1부터 100까지의 합, 1000이 넘으면 종료**

```SQL
DELIMITER $$
CREATE PROCEDURE whileProc2()
BEGIN
    DECLARE i INT; -- 1에서 100까지 증가할 지역 변수
    DECLARE hap INT; -- 더한 값을 누적할 지역 변수
    SET i = 1;
    SET hap = 0;

    myWhile: 
    WHILE (i <= 100) DO  -- While문에 label을 지정
       IF (i%4 = 0) THEN
         SET i = i + 1;     
         ITERATE myWhile; -- 지정한 label문으로 가서 계속 진행
       END IF;
       SET hap = hap + i; 
       IF (hap > 1000) THEN 
         LEAVE myWhile; -- 지정한 label문을 떠남. 즉, While 종료.
       END IF;
       SET i = i + 1;
    END WHILE;
    SELECT 'Sum from 1 to 100 (excluding multiples of 4), end when over 1000 ==>', hap; 
END $$
DELIMITER ;
```

In [233]:
dbGetQuery(conn, "CALL whileProc2(); ")

"Sum from 1 to 100 (excluding multiples of 4), ending when over 1000 ==>",hap
<chr>,<int>
"Sum from 1 to 100 (excluding multiples of 4), ending when over 1000 ==>",1014
